In [1]:
import cv2 #Video and Image processing
import numpy as np #Array manipulation

In [2]:
#load pre-trained DNN model
model_path = './res10_300x300_ssd_iter_140000.caffemodel'
config_path = './deploy.prototxt'
net = cv2.dnn.readNetFromCaffe(config_path, model_path)

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +  './haarcascade_eye.xml')

In [5]:
#detect faces and swap eyes in a given frame, but now with a DNN!
def swap_frame(frame):
    #convert the frame to grayscale for haar cascades
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300, 300), mean=(104.0, 177.0, 123.0))
    
    #face detect
    net.setInput(blob)
    detections = net.forward()
    
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype(int)
            
            #Exctract face ROI
            face_roi = frame[y1:y2, x1:x2]
            
            #apply haar cascades for eye detection in ROI
            gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
           
            detected_eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
            
            if len(detected_eyes) == 2:
                #Sort by x-coordinate
                detected_eyes = sorted(detected_eyes, key=lambda x: x[0])
                
                #swap detected eyes
                left_eye = detected_eyes[0]
                right_eye = detected_eyes[1]
                
                #extract eye regions
                lx, ly, lw, lh = left_eye
                rx, ry, rw, rh = right_eye
                
                left_eye_region = face_roi[ly:ly+lh, lx:lx+lw]
                right_eye_region = face_roi[ry:ry+rh,rx:rx+rh]
                
                #Flip ROI upside down
                #left_eye_flipped = cv2.flip(left_eye_region, 0)
                #right_eye_flipped = cv2.flip(right_eye_region, 0)
                
                #resize the eye regions to match the target areas
                left_eye_resized = cv2.resize(left_eye_region, (rw, rh))
                right_eye_resized = cv2.resize(right_eye_region, (lw, lh))
                
                #swap eyes
                face_roi[ly:ly+lh, lx:lx+lw] = right_eye_resized
                face_roi[ry:ry+rh, rx:rx+rw] = left_eye_resized
            
            #Replace face ROI back in the frame
            frame[y1:y2, x1:x2] = face_roi
    return frame
                

In [6]:
#Real-time feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print('Failed to capture frame')
        break
    
    frame = swap_frame(frame)
    cv2.imshow('Eye Swap', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()